In [2]:
from flask import Flask, request, jsonify
import requests
import pandas as pd
import json

USSD_URL = 'https://ussdapi-vv2j.onrender.com'
app = Flask(__name__)

class USSDResponse:
    def __init__(self, session_id=None, user_id=None, msisdn=None, message=None, continue_session=False):
        self.sessionID = session_id
        self.userID = user_id
        self.msisdn = msisdn
        self.message = message
        self.continueSession = continue_session

In [4]:
def get_valid_student_id():
    df = pd.read_csv('indexes.csv')
    student_list = df['student_id'].tolist()

    while True:
        try:
            student_id = int(input('Student ID: '))
            if student_id in student_list:
                return student_id
            else:
                print('Student ID not found. Please try again.')
        except ValueError:
            print('Invalid input. Please enter a numeric Student ID.')

In [5]:
def get_valid_password(student_id):
    while True:
        password = input('Password: ')
        data = {'username': student_id, 'password': password}
        try:
            response = requests.post(f'{USSD_URL}/', json=data)
            response.raise_for_status()
            return response.json().get('access_token')
        except requests.HTTPError:
            print('Password is incorrect. Please try again.')

In [ ]:
def home():
    global api_token
    home_response = requests.get(f'{USSD_URL}/home', headers={'Authorization': f'Bearer {api_token}'})
    home_response.raise_for_status()
    user_name = home_response.json().get('first_name')
    print(f"Welcome {user_name}! What would you like to do today?")

    user_response = int(input('1. My Account\n2. Check Results\n3. Register Courses\n4. Time Table\n'
                              '5. School Calendar\n6. Log Out\nEnter an option to proceed: '))

    if user_response == 1:
        account_response = requests.get(f'{USSD_URL}/account', headers={'Authorization': f'Bearer {api_token}'})
        account_response.raise_for_status()
        account_details = account_response.json()
        account(account_details)

    elif user_response == 2:
        result_response = requests.get(f'{USSD_URL}/results', headers={'Authorization': f'Bearer {api_token}'})
        result_response.raise_for_status()
        result_details = result_response.json()
        display_results(result_details)

    elif user_response == 3:
        courses_response = requests.get(f'{USSD_URL}/courses', headers={'Authorization': f'Bearer {api_token}'})
        courses_response.raise_for_status()
        courses_details = courses_response.json()
        register_courses(courses_details, api_token)

    else:
        exit()